In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import regex as re
from bs4 import BeautifulSoup
import requests
import urllib
from urllib.request import urlopen
from selenium.common.exceptions import NoSuchElementException

In [2]:
def check_if_exists(Class):
    x = driver.find_elements(By.CLASS_NAME, Class)
    if x == []:
        return False
    else:
        return True

verzoek_data lijkt niet meer te werken maar dat maakt niks meer uit want hij staat werkend in VSCode

In [96]:
def verzoek_data(url):
    
    # keys voor de dictionary
    keys=['Naam', 'Verantwoordelijke', 'Thema', 'Documentsoort', 'Publicatiedatum', 'Documentdatum', 'Onderwerp', 'url']
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    
    # check of document deel uit maakt van een ander document
    if check_if_exists('alert__inner') == True:
        
        # als dit het geval is ga verder op de nieuwe pagina
        URL_ = driver.find_element(By.CLASS_NAME, 'alert__inner')
        URL = URL_.find_element(By.TAG_NAME, 'a').get_attribute('href')
        driver.get(URL)
        
    title = (driver.find_element(By.CLASS_NAME, "h2")).text
    Row = [title]

    FILE = driver.find_element(By.CLASS_NAME, "result--list ")
    url = FILE.find_elements(By.TAG_NAME, 'a')
    
    url_list = [i.get_attribute('href') for i in url]

    # vind de tabel met data en verzamel de data hieruit
    table = driver.find_element(By.CLASS_NAME, "table__data-overview")

    for row in table.find_elements(By.XPATH, ".//tr"):
        row_data = row.find_element(By.XPATH, ".//td").get_attribute('textContent')

        # als de data van een bepaald vak een lijst is maakt deze code er een python list van
        if ('\n') in row_data:
            Data = []

            i = row_data.split('\n')
            for item in i:
                res = re.findall(r'\w+', item)
                if res != []:
                    sentence = ' '.join([str(r) for r in res])
                    Data.append(sentence)
        else:
            Data = row_data
        
        Row.append(Data)

    if len(Row) == 6:
        Row.append(np.nan)
        
    Row.append(url_list)
    
    result = dict(zip(keys, Row))
    
    return result
    
print(verzoek_data('https://open.overheid.nl/Details/ronl-b1ffb1d37c52ac6e77427cffeaf330381012b589/1?hit=4&count=50&informatiesoort=c_4f50ca9c'))
    

MaxRetryError: HTTPConnectionPool(host='localhost', port=50228): Max retries exceeded with url: /session/75c092ddc7f8e0be81bd2e84520f5fa2/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001FB8E871A60>: Failed to establish a new connection: [WinError 10061] Kan geen verbinding maken omdat de doelcomputer de verbinding actief heeft geweigerd'))

In [59]:
# installeer webdriver indien nodig en maak benodigde URL driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://open.overheid.nl/zoekresultaten?datumBeschikbaarVanaf=01-01-2023&sort=date-asc&informatiesoort=c_4f50ca9c&page=1&count=761")

# zet dataframe eerst als none zodat de columns toegevoegd kunnen worden
Dataframe = None

# de columnnamen zijn opzich ook te verzamelen door middel van Selenium, maar dat is wel onnodig.
column=['Titel', 'Verzoek', 'Specifiek', 'Verantwoordelijke', 'Thema', 'Documentsoort', 'Publicatiedatum', 'Documentdatum', 'Onderwerp', 'url']

# localiseer alle documenten
lists = driver.find_elements(By.CLASS_NAME, 'result--title ')

# vind alle URLS in de documenten
URLS = []
for i in lists:
    element = i.find_element(By.TAG_NAME, 'a').get_attribute("href")
    URLS.append(element)

# loop over al de URLS 
for i in URLS:
    usable = True
    
    # verander de driver met de nieuwe URL
    driver.get(i)
    
    # vind de titel van het document
    title = (driver.find_element(By.CLASS_NAME, "h2")).text
    
    FILE = driver.find_element(By.CLASS_NAME, "result--list ")
    url = FILE.find_elements(By.TAG_NAME, 'a')

    url_list = [i.get_attribute('href') for i in url]
    
    if len(url_list) != 1:
        usable = False
    
    # split titel in twee delen indien mogelijk
    split = title.split(" - ")
    if len(split) == 1:
        Row = [title, split[0], np.nan]
    else:
        Row = [title, split[0], split[1]]
    
    # vind de tabel met data en verzamel de data hieruit
    table = driver.find_element(By.CLASS_NAME, "table__data-overview")
    
    for row in table.find_elements(By.XPATH, ".//tr"):
        row_data = row.find_element(By.XPATH, ".//td").get_attribute('textContent')
        
        # als de data van een bepaald vak een lijst is maakt deze code er een python list van
        if ('\n') in row_data:
            Data = []
            
            i = row_data.split('\n')
            for item in i:
                res = re.findall(r'\w+', item)
                if res != []:
                    sentence = ' '.join([str(r) for r in res])
                    Data.append(sentence)
        else:
            Data = row_data
        
        Row.append(Data)
        
    # insert kolommmen en rijen in dataframe
    if Dataframe is None:
        Dataframe = pd.DataFrame(columns=column)

    if len(Row) == 8:
        Row.append(np.nan)
    
    if usable == True:
        Row.append(url_list[0])
        Dataframe.loc[len(Dataframe)] = Row
        
    # Het verzoek wat hier wordt gepasst heeft een grote kans om alsnog in de database te komen omdat er andere instanties
    # zijn waarvan de structuur wel voldoet.
    else:
        pass
    
driver.quit()

display(Dataframe)

,Titel,Verzoek,Specifiek,Verantwoordelijke,Thema,Documentsoort,Publicatiedatum,Documentdatum,Onderwerp,url
0,Besluit op Woo-verzoek over geleiderails langs...,Besluit op Woo-verzoek over geleiderails langs...,Bijlagen deel 5 bij Woo-besluit over geleidera...,Ministerie van Infrastructuur en Waterstaat,Rail- en wegverkeer,Beslissing op Wob-/Woo-verzoek,02-01-2023,23-12-2022,Wegen,https://open.overheid.nl/documenten/ronl-066ba...
1,Besluit op Woo-verzoek over geleiderails langs...,Besluit op Woo-verzoek over geleiderails langs...,Bijlagen deel 1 bij Woo-besluit over geleidera...,Ministerie van Infrastructuur en Waterstaat,Rail- en wegverkeer,Beslissing op Wob-/Woo-verzoek,02-01-2023,23-12-2022,Wegen,https://open.overheid.nl/documenten/ronl-3e22b...
2,Besluit op Woo-verzoek over geleiderails langs...,Besluit op Woo-verzoek over geleiderails langs...,Bijlagen deel 4 bij Woo-besluit over geleidera...,Ministerie van Infrastructuur en Waterstaat,Rail- en wegverkeer,Beslissing op Wob-/Woo-verzoek,02-01-2023,23-12-2022,Wegen,https://open.overheid.nl/documenten/ronl-452af...
3,Besluit op Woo-verzoek over geleiderails langs...,Besluit op Woo-verzoek over geleiderails langs...,Bijlagen deel 2 bij Woo-besluit over geleidera...,Ministerie van Infrastructuur en Waterstaat,Rail- en wegverkeer,Beslissing op Wob-/Woo-verzoek,02-01-2023,23-12-2022,Wegen,https://open.overheid.nl/documenten/ronl-58d15...
4,Besluit op Woo-verzoek over geleiderails langs...,Besluit op Woo-verzoek over geleiderails langs...,Besluit en inventaris bij Woo-verzoek over gel...,Ministerie van Infrastructuur en Waterstaat,Rail- en wegverkeer,Beslissing op Wob-/Woo-verzoek,02-01-2023,23-12-2022,Wegen,https://open.overheid.nl/documenten/ronl-836a0...
...,...,...,...,...,...,...,...,...,...,...
617,Herziening besluit op Woo-verzoek over in- en ...,Herziening besluit op Woo-verzoek over in- en ...,Documenten deel 3 bij Woo-verzoek Ameerega Sil...,"Ministerie van Landbouw, Natuur en Voedselkwal...",Natuur- en landschapsbeheer,Beslissing op Wob-/Woo-verzoek,02-03-2023,09-02-2023,Natuur en biodiversiteit,https://open.overheid.nl/documenten/ronl-70a48...
618,Herziening besluit op Woo-verzoek over in- en ...,Herziening besluit op Woo-verzoek over in- en ...,Documenten deel 2 bij Woo-verzoek Ameerega Sil...,"Ministerie van Landbouw, Natuur en Voedselkwal...",Natuur- en landschapsbeheer,Beslissing op Wob-/Woo-verzoek,02-03-2023,09-02-2023,Natuur en biodiversiteit,https://open.overheid.nl/documenten/ronl-7b5a4...
619,Herziening besluit op Woo-verzoek over in- en ...,Herziening besluit op Woo-verzoek over in- en ...,Documenten deel 6 bij Woo-verzoek Ameerega Sil...,"Ministerie van Landbouw, Natuur en Voedselkwal...",Natuur- en landschapsbeheer,Beslissing op Wob-/Woo-verzoek,02-03-2023,09-02-2023,Natuur en biodiversiteit,https://open.overheid.nl/documenten/ronl-c564c...
620,Besluit op Woo-verzoek over financiële sanctie...,Besluit op Woo-verzoek over financiële sanctie...,Documenten bij Woo-besluit over financiële san...,Ministerie van Buitenlandse Zaken,"[Europese zaken, Internationale betrekkingen]",Beslissing op Wob-/Woo-verzoek,02-03-2023,28-02-2023,"[Europese Unie, Oorlog in Oekraïne]",https://open.overheid.nl/documenten/ronl-d5e45...


In [85]:
%%time

import io
from PyPDF2 import PdfReader

# Hiermee is het mogelijk om pdf's te lezen uit een pdfreader wellicht waardeloos, maar kan in sommige gevallen wellicht 
# wel handig blijken want soms is de verzoekdatum wel makkelijk te achthttps://open.overheid.nl/Details/ronl-b1ffb1d37c52ac6e77427cffeaf330381012b589/1#panel-teksterhalen (in uw brief van ... ) om maar een vb te noemen
url = 'https://open.overheid.nl/documenten/ronl-9095dee713688ca519fdb5a82090fc5296c27cd7/pdf'

r = requests.get(url)
f = io.BytesIO(r.content)

reader = PdfReader(f)

# leest eerste pagina enige mogelijk nuttige pagina lijkt mij
contents = reader.pages[0].extract_text().split('\n')
contents = [content.strip() for content in contents]

print(contents)

['', '', '', 'Directie Juridische Zaken', '', 'Korte Voorhout 7', '2511 CW  Den Haag', 'Postbus 20201', '2500 EE  Den Haag', 'www.rijksoverheid.nl', '', 'Inlichtingen', 'www.minfin.nl', '', 'Ons kenmerk', '2022-0000284765', '', 'Uw brief (kenmerk)', '3 augustus 2022', '', 'Bijlage', 'A Relevante artikelen Woo  > Retouradres Postbus 20201 2500 EE  Den Haag', '', '', 'Pagina 1 van 4', 'Datum  27 februari 2023', 'Betreft  Besluit op uw Woo verzoek inzake informatie bilaterale', 'bijeenkomst Nederland – Rusland G20 forum 2019', '', '', '', 'Geachte ,', 'In uw verzoek van 3 augustus 2022, door mij op dezelfde dag ontvangen, heeft u mijn ministerie gevraagd informatie openbaar te maken over een bilateraal', 'gesprek tussen Nederland en Rusland op de G20 top in Osaka in 2019.', '', 'U verzoekt om de volgende document en:', '- Overwegingen, overleg, communicatie en besluitvorming van de Minister van', 'Financiën betreffende het tot stand komen van de bilaterale ontmoeting', 'tussen Nederland e

In [60]:
# Sla het op als csv file zodat dit niet nog een keer gerunt hoeft te worden. Duurde ongeveer 45 minuten namelijk
Dataframe.to_csv(r'C:\Users\ramon\Downloads\WOO2023.csv', index=False)

In [107]:
def eerstwrd(woord):
    
    eerste2woorden = woord.split(' ')[:2]
    string = ' '.join(eerste2woorden)
    if 'besluit' in string:
        Split = woord.split('besluit')
        return 'Besluit' + Split[1]
    
    elif 'beslissing' in string:
        Split = woord.split('beslissing')
        return 'Beslissing' + Split[1]
    
    else:
        return woord

In [126]:
# laad de csv als dataframe
df = pd.read_csv('WOO2023.csv')

# compute het jaar en drop documenten uit 2018
df['year'] = df['Documentdatum'].str.slice(-4, )
df = df[df['year'] == '2023']
df['Verzoek'] = df['Verzoek'].map(lambda x:eerstwrd(x))

# deze functie zal het wisselende beginwoord proberen te veranderen naar Besluit of Beslissing
df['eerste_woord'] = df['Verzoek'].str.split(' ').str.get(0)



# Omdat het moeilijk is om te bepalen welk beginwoord wel of niet te droppen heb ik besloten enkel deze te behouden.
woord_list = ['Besluit', 'Wob-verzoek', 'Beslissing', 'Wob-besluit', 'Woo-verzoek','Woo-besluit']

df = df[df['eerste_woord'].isin(woord_list)]
df = df.drop('eerste_woord', axis=1)

display(df)

,Titel,Verzoek,Specifiek,Verantwoordelijke,Thema,Documentsoort,Publicatiedatum,Documentdatum,Onderwerp,url,year
12,Besluit Woo-verzoek - geen documenten aangetro...,Besluit Woo-verzoek,geen documenten aangetroffen,Ministerie van Justitie en Veiligheid,Onbekend thema,Beslissing op Wob-/Woo-verzoek,03-01-2023,03-01-2023,NaN,https://open.overheid.nl/documenten/ronl-7e84d...,2023
20,Besluit Woo-verzoek - geen documenten aangetro...,Besluit Woo-verzoek,geen documenten aangetroffen,Ministerie van Justitie en Veiligheid,Onbekend thema,Beslissing op Wob-/Woo-verzoek,03-01-2023,03-01-2023,NaN,https://open.overheid.nl/documenten/ronl-f78aa...,2023
35,Besluit op Wob-/Woo-verzoek over correspondent...,Besluit op Wob-/Woo-verzoek over correspondent...,Bijlage 1,Ministerie van Infrastructuur en Waterstaat,Openbaar vervoer,Beslissing op Wob-/Woo-verzoek,05-01-2023,03-01-2023,Openbaar vervoer (ov),https://open.overheid.nl/documenten/ronl-3a77a...,2023
36,Besluit op Wob-/Woo-verzoek over correspondent...,Besluit op Wob-/Woo-verzoek over correspondent...,Bijlage 4,Ministerie van Infrastructuur en Waterstaat,Openbaar vervoer,Beslissing op Wob-/Woo-verzoek,05-01-2023,03-01-2023,Openbaar vervoer (ov),https://open.overheid.nl/documenten/ronl-68da5...,2023
37,Besluit op Wob-/Woo-verzoek over correspondent...,Besluit op Wob-/Woo-verzoek over correspondent...,Bijlage 3,Ministerie van Infrastructuur en Waterstaat,Openbaar vervoer,Beslissing op Wob-/Woo-verzoek,05-01-2023,03-01-2023,Openbaar vervoer (ov),https://open.overheid.nl/documenten/ronl-77381...,2023
...,...,...,...,...,...,...,...,...,...,...,...
617,Herziening besluit op Woo-verzoek over in- en ...,Besluit op Woo-verzoek over in- en uitvoerverg...,Documenten deel 3 bij Woo-verzoek Ameerega Sil...,"Ministerie van Landbouw, Natuur en Voedselkwal...",Natuur- en landschapsbeheer,Beslissing op Wob-/Woo-verzoek,02-03-2023,09-02-2023,Natuur en biodiversiteit,https://open.overheid.nl/documenten/ronl-70a48...,2023
618,Herziening besluit op Woo-verzoek over in- en ...,Besluit op Woo-verzoek over in- en uitvoerverg...,Documenten deel 2 bij Woo-verzoek Ameerega Sil...,"Ministerie van Landbouw, Natuur en Voedselkwal...",Natuur- en landschapsbeheer,Beslissing op Wob-/Woo-verzoek,02-03-2023,09-02-2023,Natuur en biodiversiteit,https://open.overheid.nl/documenten/ronl-7b5a4...,2023
619,Herziening besluit op Woo-verzoek over in- en ...,Besluit op Woo-verzoek over in- en uitvoerverg...,Documenten deel 6 bij Woo-verzoek Ameerega Sil...,"Ministerie van Landbouw, Natuur en Voedselkwal...",Natuur- en landschapsbeheer,Beslissing op Wob-/Woo-verzoek,02-03-2023,09-02-2023,Natuur en biodiversiteit,https://open.overheid.nl/documenten/ronl-c564c...,2023
620,Besluit op Woo-verzoek over financiële sanctie...,Besluit op Woo-verzoek over financiële sanctie...,Documenten bij Woo-besluit over financiële san...,Ministerie van Buitenlandse Zaken,"['Europese zaken', 'Internationale betrekkingen']",Beslissing op Wob-/Woo-verzoek,02-03-2023,28-02-2023,"['Europese Unie', 'Oorlog in Oekraïne']",https://open.overheid.nl/documenten/ronl-d5e45...,2023


In [127]:
import math

def datum_naar_kwartaal(datum):
    """" Functie om datum om te zetten naar kwartaal"""
    
    maand = int(datum[3:5])
    kwartaal = math.ceil(maand / 3)
    return f"{kwartaal}e Kwartaal"
    
df['kwartaal'] = df['Publicatiedatum'].map(datum_naar_kwartaal)
display(df)

,Titel,Verzoek,Specifiek,Verantwoordelijke,Thema,Documentsoort,Publicatiedatum,Documentdatum,Onderwerp,url,year,kwartaal
12,Besluit Woo-verzoek - geen documenten aangetro...,Besluit Woo-verzoek,geen documenten aangetroffen,Ministerie van Justitie en Veiligheid,Onbekend thema,Beslissing op Wob-/Woo-verzoek,03-01-2023,03-01-2023,NaN,https://open.overheid.nl/documenten/ronl-7e84d...,2023,1e Kwartaal
20,Besluit Woo-verzoek - geen documenten aangetro...,Besluit Woo-verzoek,geen documenten aangetroffen,Ministerie van Justitie en Veiligheid,Onbekend thema,Beslissing op Wob-/Woo-verzoek,03-01-2023,03-01-2023,NaN,https://open.overheid.nl/documenten/ronl-f78aa...,2023,1e Kwartaal
35,Besluit op Wob-/Woo-verzoek over correspondent...,Besluit op Wob-/Woo-verzoek over correspondent...,Bijlage 1,Ministerie van Infrastructuur en Waterstaat,Openbaar vervoer,Beslissing op Wob-/Woo-verzoek,05-01-2023,03-01-2023,Openbaar vervoer (ov),https://open.overheid.nl/documenten/ronl-3a77a...,2023,1e Kwartaal
36,Besluit op Wob-/Woo-verzoek over correspondent...,Besluit op Wob-/Woo-verzoek over correspondent...,Bijlage 4,Ministerie van Infrastructuur en Waterstaat,Openbaar vervoer,Beslissing op Wob-/Woo-verzoek,05-01-2023,03-01-2023,Openbaar vervoer (ov),https://open.overheid.nl/documenten/ronl-68da5...,2023,1e Kwartaal
37,Besluit op Wob-/Woo-verzoek over correspondent...,Besluit op Wob-/Woo-verzoek over correspondent...,Bijlage 3,Ministerie van Infrastructuur en Waterstaat,Openbaar vervoer,Beslissing op Wob-/Woo-verzoek,05-01-2023,03-01-2023,Openbaar vervoer (ov),https://open.overheid.nl/documenten/ronl-77381...,2023,1e Kwartaal
...,...,...,...,...,...,...,...,...,...,...,...,...
617,Herziening besluit op Woo-verzoek over in- en ...,Besluit op Woo-verzoek over in- en uitvoerverg...,Documenten deel 3 bij Woo-verzoek Ameerega Sil...,"Ministerie van Landbouw, Natuur en Voedselkwal...",Natuur- en landschapsbeheer,Beslissing op Wob-/Woo-verzoek,02-03-2023,09-02-2023,Natuur en biodiversiteit,https://open.overheid.nl/documenten/ronl-70a48...,2023,1e Kwartaal
618,Herziening besluit op Woo-verzoek over in- en ...,Besluit op Woo-verzoek over in- en uitvoerverg...,Documenten deel 2 bij Woo-verzoek Ameerega Sil...,"Ministerie van Landbouw, Natuur en Voedselkwal...",Natuur- en landschapsbeheer,Beslissing op Wob-/Woo-verzoek,02-03-2023,09-02-2023,Natuur en biodiversiteit,https://open.overheid.nl/documenten/ronl-7b5a4...,2023,1e Kwartaal
619,Herziening besluit op Woo-verzoek over in- en ...,Besluit op Woo-verzoek over in- en uitvoerverg...,Documenten deel 6 bij Woo-verzoek Ameerega Sil...,"Ministerie van Landbouw, Natuur en Voedselkwal...",Natuur- en landschapsbeheer,Beslissing op Wob-/Woo-verzoek,02-03-2023,09-02-2023,Natuur en biodiversiteit,https://open.overheid.nl/documenten/ronl-c564c...,2023,1e Kwartaal
620,Besluit op Woo-verzoek over financiële sanctie...,Besluit op Woo-verzoek over financiële sanctie...,Documenten bij Woo-besluit over financiële san...,Ministerie van Buitenlandse Zaken,"['Europese zaken', 'Internationale betrekkingen']",Beslissing op Wob-/Woo-verzoek,02-03-2023,28-02-2023,"['Europese Unie', 'Oorlog in Oekraïne']",https://open.overheid.nl/documenten/ronl-d5e45...,2023,1e Kwartaal


In [128]:
from collections import Counter

words = []

for i in df.Verzoek:
    i = i.split(' ', 1)
    words.append(i[0])
        
X = Counter(words).most_common()

print(X)

[('Besluit', 399), ('Beslissing', 26), ('Woo-verzoek', 3)]


In [129]:
titels = set()
for i in df.Verzoek:
    titels.add(i)
    
print('Titels:', len(titels))

eerste_woord = []
for i in titels:
    x = i.split(' ')
    eerste_woord.append(x[0])
    
print(Counter(eerste_woord).most_common())


Titels: 162
[('Besluit', 153), ('Beslissing', 8), ('Woo-verzoek', 1)]


In [137]:
Data1 = df.copy()
Data1 = Data1.drop_duplicates(subset = ['Verzoek'])
Data1 = Data1.groupby(['Verantwoordelijke'])['Verantwoordelijke'].count()

display(Data1)

Data1.to_csv(r'C:\Users\ramon\Downloads\TestPlot.csv')

Verantwoordelijke
Ministerie van Binnenlandse Zaken en Koninkrijksrelaties    21
Ministerie van Buitenlandse Zaken                            9
Ministerie van Defensie                                      6
Ministerie van Economische Zaken en Klimaat                 14
Ministerie van Financiën                                    28
Ministerie van Infrastructuur en Waterstaat                 10
Ministerie van Justitie en Veiligheid                       15
Ministerie van Landbouw, Natuur en Voedselkwaliteit         45
Ministerie van Onderwijs, Cultuur en Wetenschap              6
Ministerie van Sociale Zaken en Werkgelegenheid              6
Ministerie van Volksgezondheid, Welzijn en Sport             2
Name: Verantwoordelijke, dtype: int64

In [140]:
import plotly.express as px

# simpel plotje die al met de huidige data te maken is

fig = px.bar(y=Data1.index, x=Data1.values, text_auto='1s',
            title="Aantal WOO/WOB verzoeken per Ministerie")
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

Helaas is het natuurlijk moeilijk om data te verkrijgen met verzoekdata of dat verzoeken wel of niet geslaagd zijn en met welke reden indien het niet is geslaagd. Alhoewel er natuurlijk wel test-data gemaakt kan en zal worden!